<a href="https://colab.research.google.com/github/graemet-umich/Neural_Network_Charity_Analysis/blob/main/AlphabetSoupCharity_Optimzation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Deliverable 3: Optimize the Model

#### Preprocessing the Data for a Neural Network

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.

# Colab (Colab runs 3-4 times slower than local iMac 3.3 GHz 6-Core Intel Core i5)
# from google.colab import files
# uploaded = files.upload()
# import io
# application_df = pd.read_csv(io.BytesIO(uploaded['charity_data.csv']))

# local
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
print(f'model shape: {application_df.shape}')
      
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], 1)
print(f'model shape: {application_df.shape}')
      
# save this df state for later attempts to maximize accuracy, so don't need to pd.read.csv again
application0_df = application_df.copy()

2022-11-05 00:03:49.949860: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


model shape: (34299, 12)
model shape: (34299, 10)


/Users/graemet/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:21: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


In [2]:
# Determine the number of unique values in each column.
print('The number of unique values in each column:')
print(application_df[application_df.columns].nunique())

# Look at APPLICATION_TYPE value counts for binning
app_type_val_counts = application_df.APPLICATION_TYPE.value_counts()
print('-----\n\nOriginal APPLICATION_TYPE unique counts:')
print(app_type_val_counts)

# Visualize the value counts of APPLICATION_TYPE
# app_type_val_counts.plot.density()

# Determine which values to replace if counts are less than ...?
replace_application = list(app_type_val_counts[app_type_val_counts < 528].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
print('-----\n\nAPPLICATION_TYPE unique counts after Other binning (< 528):')
print(application_df.APPLICATION_TYPE.value_counts())

The number of unique values in each column:
APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64
-----

Original APPLICATION_TYPE unique counts:
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64
-----

APPLICATION_TYPE unique counts after Other binning (< 528):
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64


In [3]:
# Look at CLASSIFICATION value counts for binning
classification_val_counts = application_df.CLASSIFICATION.value_counts()
default_max_rows = pd.options.display.max_rows
pd.set_option("display.max_rows", None)
print(classification_val_counts)
pd.set_option("display.max_rows", default_max_rows)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
C2190        1
C4200        1
C2600        1
C5200        1
C1370        1
C1248        1
C6100        1
C1820        1
C1900        1
C1236        1
C3700        1
C2570        1
C1580        1
C1245        1
C2500        1
C1570        1
C1283        1
C2380        1
C1732        1
C1728        1
C2170        1
C4120     

### Deliverable 3: Attempt 1
#### Run deliverable 1 & 2 features with more layers

In [4]:
# Determine which values to replace if counts are less than ..?
replace_class = list(classification_val_counts[classification_val_counts < 1883].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

#### Finish pre-processing

In [5]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
print(f'One-Hot encoded DataFrame shape: {encode_df.shape}')
# encode_df.head()

# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
print(f'Merged/dropped application_df shape: {application_df.shape}')
# application_df.head()

# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

One-Hot encoded DataFrame shape: (34299, 41)
Merged/dropped application_df shape: (34299, 44)


/Users/graemet/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/graemet/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/Users/graemet/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


#### Compile the model

#### Optimize model with an automated neural net
Returns the number of layers and the number of neurons in each layer.

In [6]:
# Requirement already satisfied: keras-tuner in
# /Users/graemet/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages (1.1.3)
!pip install keras-tuner

In [7]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])

# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu', 'tanh', 'sigmoid']) # 'relu','tanh','sigmoid'
    
    # Allow kerastuner to decide number of neurons in first layer ## TRY up to 100 ##
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=20,
        max_value=100,
        step=5), activation=activation, input_dim=number_input_features))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=4,
            max_value=30, ## TRY up to 30 ##
            step=2),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [8]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    seed=78,
    overwrite=True,
    max_epochs=20,
    hyperband_iterations=8,
    directory='anno025')

# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 240 Complete [00h 00m 18s]
val_accuracy: 0.7267638444900513

Best val_accuracy So Far: 0.72967928647995
Total elapsed time: 00h 28m 48s
INFO:tensorflow:Oracle triggered exit


In [9]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
print(best_hyper.values)

# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

{'activation': 'relu', 'first_units': 25, 'num_layers': 3, 'units_0': 22, 'units_1': 24, 'units_2': 26, 'units_3': 8, 'units_4': 10, 'tuner/epochs': 20, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
268/268 - 0s - loss: 0.5511 - accuracy: 0.7297 - 278ms/epoch - 1ms/step
Loss: 0.551053524017334, Accuracy: 0.72967928647995


#### Results for attempt 1

hyperband_iterations=2 (60 trials, 6'54"):

{'activation': 'relu', 'first_units': 25, 'num_layers': 3, 'units_0': 26, 'units_1': 20, 'units_2': 18, 'units_3': 10, 'units_4': 14, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0044'}
268/268 - 0s - loss: 0.5515 - accuracy: 0.7296 - 271ms/epoch - 1ms/step
Loss: 0.5515058636665344, Accuracy: 0.7295626997947693

---

hyperband_iterations=4 (120 trials, 14'47", anno24):

{'activation': 'relu', 'first_units': 75, 'num_layers': 2, 'units_0': 12, 'units_1': 24, 'units_2': 14, 'units_3': 24, 'units_4': 12, 'tuner/epochs': 20, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
268/268 - 0s - loss: 0.5523 - accuracy: 0.7291 - 316ms/epoch - 1ms/step
Loss: 0.5522750616073608, Accuracy: 0.7290962338447571

---

hyperband_iterations=8 (240 trials, 29'32", anno25):

{'activation': 'relu', 'first_units': 60, 'num_layers': 4, 'units_0': 10, 'units_1': 24, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 2, 'tuner/round': 2, 'units_2': 4, 'units_3': 4, 'tuner/trial_id': '0012'}
268/268 - 0s - loss: 0.5540 - accuracy: 0.7292 - 274ms/epoch - 1ms/step
Loss: 0.5539767742156982, Accuracy: 0.7292128205299377

In [34]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# need numpy ceil function to calculate number of batches
import numpy as np

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])

# taken from automated neural network optimizer above
hidden_nodes_layer1 = 25
hidden_nodes_layer2 = 22
hidden_nodes_layer3 = 24
hidden_nodes_layer4 = 26
hidden_nodes_layer5 = 8
hidden_nodes_layer6 = 10

nn = tf.keras.models.Sequential()
activation = "relu"

# First hidden layer (first_units)
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation=activation)
)

# Second hidden layer (units_0)
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation=activation))

# Third hidden layer (units_1)
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation=activation))

# Fourth hidden layer (units_2)
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation=activation))

# Fifth hidden layer (units_3)
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation=activation))

# Sixth hidden layer (units_4)
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer6, activation=activation))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
print(nn.summary())

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

#### Use checkpoints

# Define the checkpoint path and filenames
checkpoints_dir = f"checkpoints_attempt1_layer6"
print(checkpoints_dir)
os.makedirs(checkpoints_dir, exist_ok=True)
# checkpoint_path = "{checkpoints_dir}/weights.{epoch:03d}.hdf5"
checkpoint_path = os.path.join(checkpoints_dir, "weights.{epoch:03d}.hdf5")

# calculate number of batches in 5 epochs
# default batch_size = 32 (https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit)
batch_size = 32
# round up number of batches
# (https://stackoverflow.com/questions/54159034/what-if-the-sample-size-is-not-divisible-by-batch-size-in-keras-model)
# Note: n_batches here is 804, which is the number displayed in the left column
# under each epoch in nn.fit() results display.
n_batches = int(np.ceil(len(X_train) / batch_size)) # np.ceil() return dtype float64
# n_batches = np.empty_like(n_batches, dtype=np.int32) # cast to int64
n_batches_per_5epochs = 5 * n_batches

# Create a callback that saves the model's weights every 5 epochs
# For non-deprecated way, pass an integer number of batches to save_freq
# (see https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint)
# For deprecated way, see https://stackoverflow.com/questions/59069058/save-model-every-10-epochs-tensorflow-keras-v2
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    # non-deprecated way:
    save_freq=n_batches_per_5epochs
    # deprecated way: (easier)
    ## save_freq='epoch',
    # period=5
)
# Note: Both ways create a checkpoint folder containing identical files:
# weights.0005.hdf5 weights.0010.hdf5 ... weights.0095.hdf5 weights.0100.hdf5


Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_248 (Dense)           (None, 25)                1100      
                                                                 
 dense_249 (Dense)           (None, 22)                572       
                                                                 
 dense_250 (Dense)           (None, 24)                552       
                                                                 
 dense_251 (Dense)           (None, 26)                650       
                                                                 
 dense_252 (Dense)           (None, 8)                 216       
                                                                 
 dense_253 (Dense)           (None, 10)                90        
                                                                 
 dense_254 (Dense)           (None, 1)               

In [35]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 848us/step - loss: 304.7392 - accuracy: 0.5117
Epoch 2/100
804/804 [==============================] - 1s 854us/step - loss: 0.6918 - accuracy: 0.5321
Epoch 3/100
804/804 [==============================] - 1s 850us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 4/100
804/804 [==============================] - 1s 850us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 5/100
769/804 [===========================>..] - ETA: 0s - loss: 0.6911 - accuracy: 0.5325
Epoch 5: saving model to checkpoints_attempt1_layer6/weights.005.hdf5
804/804 [==============================] - 1s 869us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 6/100
804/804 [==============================] - 1s 845us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 7/100
804/804 [==============================] - 1s 910us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 8/100
804/804 [==============================] - 1s 842us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 9/100
804/804

804/804 [==============================] - 1s 874us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 62/100
804/804 [==============================] - 1s 896us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 63/100
804/804 [==============================] - 1s 859us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 64/100
804/804 [==============================] - 1s 858us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 65/100
775/804 [===========================>..] - ETA: 0s - loss: 0.6912 - accuracy: 0.5318
Epoch 65: saving model to checkpoints_attempt1_layer6/weights.065.hdf5
804/804 [==============================] - 1s 863us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 66/100
804/804 [==============================] - 1s 835us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 67/100
804/804 [==============================] - 1s 841us/step - loss: 0.6912 - accuracy: 0.5321
Epoch 68/100
804/804 [==============================] - 1s 838us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 69/100
804/804 [===

In [36]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6935 - accuracy: 0.5269 - 235ms/epoch - 878us/step
Loss: 0.6935015916824341, Accuracy: 0.5268804430961609


#### Refine model by testing different activation functions for each hidden layer
These results not included in study.

In [10]:
# # Import checkpoint dependencies
# import os
# from tensorflow.keras.callbacks import ModelCheckpoint

# # need numpy ceil function to calculate number of batches
# import numpy as np

# # Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# number_input_features = len(X_train[0])
# hidden_nodes_layer1 = 25
# hidden_nodes_layer2 = 26
# hidden_nodes_layer3 = 20
# hidden_nodes_layer4 = 18

# activations = ['relu', 'tanh', 'sigmoid']
# for activation1 in activations:
#     nn1 = tf.keras.models.Sequential()

#     # First hidden layer (first_units)
#     nn1.add(
#         tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation=activation1)
#     )

#     for activation2 in activations:
#         nn2 = tf.keras.models.clone_model(nn1)
        
#         # Second hidden layer (units_0)
#         nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation=activation2))

#         for activation3 in activations:
#             nn3 = tf.keras.models.clone_model(nn2)

#             # Third hidden layer (units_0)
#             nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation=activation3))

#             for activation4 in activations:
#                 nn4 = tf.keras.models.clone_model(nn3)

#                 # Fourth hidden layer (units_0)
#                 nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation=activation4))

#                 # Output layer
#                 nn4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

#                 # Check the structure of the model
#                 nn4.summary()

#                 # Compile the model
#                 nn4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

#                 #### Use checkpoints

#                 # Define the checkpoint path and filenames
#                 checkpoints_dir = f"checkpoints_attempt1_{activation1[0]}{activation2[0]}{activation3[0]}{activation4[0]}"
#                 print(checkpoints_dir)
#                 os.makedirs(checkpoints_dir, exist_ok=True)
#                 # checkpoint_path = "{checkpoints_dir}/weights.{epoch:03d}.hdf5"
#                 checkpoint_path = os.path.join(checkpoints_dir, "weights.{epoch:03d}.hdf5")

#                 # calculate number of batches in 5 epochs
#                 # default batch_size = 32 (https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit)
#                 batch_size = 32
#                 # round up number of batches
#                 # (https://stackoverflow.com/questions/54159034/what-if-the-sample-size-is-not-divisible-by-batch-size-in-keras-model)
#                 # Note: n_batches here is 804, which is the number displayed in the left column
#                 # under each epoch in nn.fit() results display.
#                 n_batches = int(np.ceil(len(X_train) / batch_size)) # np.ceil() return dtype float64
#                 # n_batches = np.empty_like(n_batches, dtype=np.int32) # cast to int64
#                 n_batches_per_5epochs = 5 * n_batches

#                 # Create a callback that saves the model's weights every 5 epochs
#                 # For non-deprecated way, pass an integer number of batches to save_freq
#                 # (see https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint)
#                 # For deprecated way, see https://stackoverflow.com/questions/59069058/save-model-every-10-epochs-tensorflow-keras-v2
#                 cp_callback = ModelCheckpoint(
#                     filepath=checkpoint_path,
#                     verbose=1,
#                     save_weights_only=True,
#                     # non-deprecated way:
#                     save_freq=n_batches_per_5epochs
#                     # deprecated way: (easier)
#                     ## save_freq='epoch',
#                     # period=5
#                 )
#                 # Note: Both ways create a checkpoint folder containing identical files:
#                 # weights.0005.hdf5 weights.0010.hdf5 ... weights.0095.hdf5 weights.0100.hdf5

#                 stop_early_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

#                 # Train the model
#                 fit_model = nn4.fit(X_train, y_train, epochs=100, callbacks=[cp_callback])

#                 # Evaluate the model using the test data
#                 model_loss, model_accuracy = nn4.evaluate(X_test_scaled,y_test,verbose=2)
#                 print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 25)                1100      
                                                                 
 dense_6 (Dense)             (None, 26)                676       
                                                                 
 dense_7 (Dense)             (None, 20)                540       
                                                                 
 dense_8 (Dense)             (None, 18)                378       
                                                                 
 dense_9 (Dense)             (None, 1)                 19        
                                                                 
Total params: 2,713
Trainable params: 2,713
Non-trainable params: 0
_________________________________________________________________
checkpoints_attempt1_rrrr
Epoch 1/100
804/804 [=====

804/804 [==============================] - 1s 786us/step - loss: 0.6910 - accuracy: 0.5321
Epoch 55/100
775/804 [===========================>..] - ETA: 0s - loss: 0.6913 - accuracy: 0.5315
Epoch 55: saving model to checkpoints_attempt1_rrrr/weights.055.hdf5
804/804 [==============================] - 1s 794us/step - loss: 0.6912 - accuracy: 0.5321
Epoch 56/100
804/804 [==============================] - 1s 779us/step - loss: 0.6912 - accuracy: 0.5321
Epoch 57/100
804/804 [==============================] - 1s 783us/step - loss: 0.6913 - accuracy: 0.5321
Epoch 58/100
804/804 [==============================] - 1s 792us/step - loss: 0.6912 - accuracy: 0.5321
Epoch 59/100
804/804 [==============================] - 1s 802us/step - loss: 0.6912 - accuracy: 0.5321
Epoch 60/100
779/804 [============================>.] - ETA: 0s - loss: 0.6913 - accuracy: 0.5321
Epoch 60: saving model to checkpoints_attempt1_rrrr/weights.060.hdf5
804/804 [==============================] - 1s 790us/step - loss: 0.6

804/804 [==============================] - 1s 962us/step - loss: 0.6916 - accuracy: 0.5308
Epoch 6/100
804/804 [==============================] - 1s 952us/step - loss: 0.6914 - accuracy: 0.5321
Epoch 7/100
804/804 [==============================] - 1s 941us/step - loss: 0.6916 - accuracy: 0.5314
Epoch 8/100
804/804 [==============================] - 1s 796us/step - loss: 0.6914 - accuracy: 0.5311
Epoch 9/100
804/804 [==============================] - 1s 795us/step - loss: 0.6914 - accuracy: 0.5321
Epoch 10/100
765/804 [===========================>..] - ETA: 0s - loss: 0.6915 - accuracy: 0.5324
Epoch 10: saving model to checkpoints_attempt1_rrrt/weights.010.hdf5
804/804 [==============================] - 1s 803us/step - loss: 0.6916 - accuracy: 0.5314
Epoch 11/100
804/804 [==============================] - 1s 795us/step - loss: 0.6916 - accuracy: 0.5316
Epoch 12/100
804/804 [==============================] - 1s 803us/step - loss: 0.6913 - accuracy: 0.5313
Epoch 13/100
804/804 [=========

804/804 [==============================] - 1s 794us/step - loss: 0.6913 - accuracy: 0.5306
Epoch 67/100
804/804 [==============================] - 1s 799us/step - loss: 0.6914 - accuracy: 0.5311
Epoch 68/100
804/804 [==============================] - 1s 795us/step - loss: 0.6916 - accuracy: 0.5321
Epoch 69/100
804/804 [==============================] - 1s 796us/step - loss: 0.6916 - accuracy: 0.5315
Epoch 70/100
794/804 [============================>.] - ETA: 0s - loss: 0.6915 - accuracy: 0.5323
Epoch 70: saving model to checkpoints_attempt1_rrrt/weights.070.hdf5
804/804 [==============================] - 1s 914us/step - loss: 0.6915 - accuracy: 0.5321
Epoch 71/100
804/804 [==============================] - 1s 934us/step - loss: 0.6915 - accuracy: 0.5321
Epoch 72/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6914 - accuracy: 0.5309
Epoch 73/100
804/804 [==============================] - 1s 849us/step - loss: 0.6916 - accuracy: 0.5309
Epoch 74/100
804/804 [=======

804/804 [==============================] - 1s 797us/step - loss: 0.6914 - accuracy: 0.5321
Epoch 19/100
804/804 [==============================] - 1s 796us/step - loss: 0.6913 - accuracy: 0.5321
Epoch 20/100
765/804 [===========================>..] - ETA: 0s - loss: 0.6914 - accuracy: 0.5320
Epoch 20: saving model to checkpoints_attempt1_rrrs/weights.020.hdf5
804/804 [==============================] - 1s 804us/step - loss: 0.6914 - accuracy: 0.5321
Epoch 21/100
804/804 [==============================] - 1s 801us/step - loss: 0.6915 - accuracy: 0.5321
Epoch 22/100
804/804 [==============================] - 1s 797us/step - loss: 0.6913 - accuracy: 0.5321
Epoch 23/100
804/804 [==============================] - 1s 795us/step - loss: 0.6913 - accuracy: 0.5321
Epoch 24/100
804/804 [==============================] - 1s 796us/step - loss: 0.6914 - accuracy: 0.5321
Epoch 25/100
752/804 [===========================>..] - ETA: 0s - loss: 0.6914 - accuracy: 0.5319
Epoch 25: saving model to checkpo

...

804/804 [==============================] - 1s 838us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 56/100
804/804 [==============================] - 1s 843us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 57/100
804/804 [==============================] - 1s 805us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 58/100
804/804 [==============================] - 1s 805us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 59/100
804/804 [==============================] - 1s 804us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 60/100
759/804 [===========================>..] - ETA: 0s - loss: 0.6911 - accuracy: 0.5319
Epoch 60: saving model to checkpoints_attempt1_sssr/weights.060.hdf5
804/804 [==============================] - 1s 811us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 61/100
804/804 [==============================] - 1s 797us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 62/100
804/804 [==============================] - 1s 797us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 63/100
804/804 [=====

804/804 [==============================] - 1s 801us/step - loss: 0.6917 - accuracy: 0.5291
Epoch 8/100
804/804 [==============================] - 1s 800us/step - loss: 0.6915 - accuracy: 0.5316
Epoch 9/100
804/804 [==============================] - 1s 877us/step - loss: 0.6915 - accuracy: 0.5304
Epoch 10/100
795/804 [============================>.] - ETA: 0s - loss: 0.6914 - accuracy: 0.5308
Epoch 10: saving model to checkpoints_attempt1_ssst/weights.010.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6914 - accuracy: 0.5307
Epoch 11/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6915 - accuracy: 0.5321
Epoch 12/100
804/804 [==============================] - 1s 984us/step - loss: 0.6914 - accuracy: 0.5321
Epoch 13/100
804/804 [==============================] - 1s 936us/step - loss: 0.6914 - accuracy: 0.5321
Epoch 14/100
804/804 [==============================] - 1s 856us/step - loss: 0.6915 - accuracy: 0.5321
Epoch 15/100
794/804 [===========

804/804 [==============================] - 1s 805us/step - loss: 0.6913 - accuracy: 0.5321
Epoch 69/100
804/804 [==============================] - 1s 799us/step - loss: 0.6913 - accuracy: 0.5321
Epoch 70/100
761/804 [===========================>..] - ETA: 0s - loss: 0.6914 - accuracy: 0.5323
Epoch 70: saving model to checkpoints_attempt1_ssst/weights.070.hdf5
804/804 [==============================] - 1s 808us/step - loss: 0.6914 - accuracy: 0.5321
Epoch 71/100
804/804 [==============================] - 1s 847us/step - loss: 0.6913 - accuracy: 0.5321
Epoch 72/100
804/804 [==============================] - 1s 799us/step - loss: 0.6913 - accuracy: 0.5321
Epoch 73/100
804/804 [==============================] - 1s 838us/step - loss: 0.6913 - accuracy: 0.5321
Epoch 74/100
804/804 [==============================] - 1s 949us/step - loss: 0.6913 - accuracy: 0.5321
Epoch 75/100
751/804 [===========================>..] - ETA: 0s - loss: 0.6913 - accuracy: 0.5313
Epoch 75: saving model to checkpo

751/804 [===========================>..] - ETA: 0s - loss: 0.6914 - accuracy: 0.5322
Epoch 20: saving model to checkpoints_attempt1_ssss/weights.020.hdf5
804/804 [==============================] - 1s 818us/step - loss: 0.6914 - accuracy: 0.5321
Epoch 21/100
804/804 [==============================] - 1s 828us/step - loss: 0.6915 - accuracy: 0.5314
Epoch 22/100
804/804 [==============================] - 1s 809us/step - loss: 0.6915 - accuracy: 0.5321
Epoch 23/100
804/804 [==============================] - 1s 826us/step - loss: 0.6913 - accuracy: 0.5321
Epoch 24/100
804/804 [==============================] - 1s 818us/step - loss: 0.6914 - accuracy: 0.5321
Epoch 25/100
803/804 [============================>.] - ETA: 0s - loss: 0.6913 - accuracy: 0.5321
Epoch 25: saving model to checkpoints_attempt1_ssss/weights.025.hdf5
804/804 [==============================] - 1s 827us/step - loss: 0.6913 - accuracy: 0.5321
Epoch 26/100
804/804 [==============================] - 1s 812us/step - loss: 0.6

804/804 [==============================] - 1s 837us/step - loss: 0.6914 - accuracy: 0.5321
Epoch 81/100
804/804 [==============================] - 1s 807us/step - loss: 0.6914 - accuracy: 0.5321
Epoch 82/100
804/804 [==============================] - 1s 817us/step - loss: 0.6912 - accuracy: 0.5321
Epoch 83/100
804/804 [==============================] - 1s 810us/step - loss: 0.6913 - accuracy: 0.5321
Epoch 84/100
804/804 [==============================] - 1s 828us/step - loss: 0.6912 - accuracy: 0.5321
Epoch 85/100
755/804 [===========================>..] - ETA: 0s - loss: 0.6911 - accuracy: 0.5331
Epoch 85: saving model to checkpoints_attempt1_ssss/weights.085.hdf5
804/804 [==============================] - 1s 813us/step - loss: 0.6912 - accuracy: 0.5321
Epoch 86/100
804/804 [==============================] - 1s 813us/step - loss: 0.6913 - accuracy: 0.5321
Epoch 87/100
804/804 [==============================] - 1s 815us/step - loss: 0.6913 - accuracy: 0.5321
Epoch 88/100
804/804 [=====

#### Results

rrrr:
268/268 - 0s - loss: 0.6984 - accuracy: 0.5331 - 227ms/epoch - 847us/step
Loss: 0.6983901858329773, Accuracy: 0.5330612063407898

Epoch 100: saving model to checkpoints_attempt1_rrrt/weights.100.hdf5
804/804 [==============================] - 1s 867us/step - loss: 0.6911 - accuracy: 0.5321
268/268 - 0s - loss: 0.6910 - accuracy: 0.5332 - 240ms/epoch - 895us/step
Loss: 0.6909899711608887, Accuracy: 0.5331778526306152
Model: "sequential_13"

Epoch 100: saving model to checkpoints_attempt1_rrrs/weights.100.hdf5
804/804 [==============================] - 1s 927us/step - loss: 0.6914 - accuracy: 0.5321
268/268 - 0s - loss: 0.6910 - accuracy: 0.5332 - 251ms/epoch - 938us/step
Loss: 0.6910285949707031, Accuracy: 0.5331778526306152
Model: "sequential_13"



### Deliverable 3: Attempt 2
#### Creating more bins for rare occurrences in columns : Expand CLASSIFICATION feature number from 6 to 7.
When considering rare occurrences in CLASSIFICATION values, 
check how much greater a value count is than the next lower value count:

| Value | Count | How much greater |
| :--- | ---: | ---: |
|C2100 | 1883 | 1883 / 777 = 2.42 |
|C7000 |  777 | 777 / 287 = 2.71 |
|C1700 |  287 | 287 / 194 = 1.48 |
|C4000 |  194 |  |

It is reasonable to try to include C7000 in its own bin as an added feature. 

This condition is not the case for the above binning of APPLICATION_TYPE values.

In [48]:
# read original dataframe with identifier fields already dropped
application_df = application0_df.copy()

# Determine which values to replace if counts are less than ..?
replace_class = list(classification_val_counts[classification_val_counts < 528].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [49]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
print(f"encode_df.shape: {encode_df.shape}")

# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
print(f"application_df.shape: {application_df.shape}")

# Split our preprocessed data into our features and target arrays
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Deliverable 2: Compile, Train and Evaluate the Model

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# # Define the checkpoint path and filenames
# os.makedirs("checkpoints_attempt2/",exist_ok=True)
# checkpoint_path = "checkpoints_opt2/weights.{epoch:04d}.hdf5"

# Define the checkpoint path and filenames
checkpoints_dir = f"checkpoints_attempt2_layer2"
print(checkpoints_dir)
os.makedirs(checkpoints_dir, exist_ok=True)
# checkpoint_path = "{checkpoints_dir}/weights.{epoch:03d}.hdf5"
checkpoint_path = os.path.join(checkpoints_dir, "weights.{epoch:03d}.hdf5")

# need numpy ceil function
import numpy as np

# calculate number of batches in 5 epochs
# default batch_size = 32 (https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit)
batch_size = 32
# round up number of batches
# (https://stackoverflow.com/questions/54159034/what-if-the-sample-size-is-not-divisible-by-batch-size-in-keras-model)
# Note: n_batches here is 804, which is the number displayed in the left column
# under each epoch in nn.fit() results display.
n_batches = int(np.ceil(len(X_train) / batch_size)) # np.ceil() return dtype float64
# n_batches = np.empty_like(n_batches, dtype=np.int32) # cast to int64
n_batches_per_5epochs = 5 * n_batches

# Create a callback that saves the model's weights every 5 epochs
# For non-deprecated way, pass an integer number of batches to save_freq
# (see https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint)
# For deprecated way, see https://stackoverflow.com/questions/59069058/save-model-every-10-epochs-tensorflow-keras-v2
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    # non-deprecated way:
    save_freq=n_batches_per_5epochs
    # deprecated way: (easier)
    ## save_freq='epoch',
    # period=5
)
# Note: Both ways create a checkpoint folder containing identical files:
# weights.0005.hdf5 weights.0010.hdf5 ... weights.0095.hdf5 weights.0100.hdf5

# Train the model
fit_model = nn.fit(X_train, y_train, epochs=100, callbacks=[cp_callback])

encode_df.shape: (34299, 50)
application_df.shape: (34299, 53)
Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_261 (Dense)           (None, 80)                4240      
                                                                 
 dense_262 (Dense)           (None, 30)                2430      
                                                                 
 dense_263 (Dense)           (None, 1)                 31        
                                                                 
Total params: 6,701
Trainable params: 6,701
Non-trainable params: 0
_________________________________________________________________
checkpoints_attempt2_layer2
Epoch 1/100


/Users/graemet/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/graemet/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  app.launch_new_instance()
/Users/graemet/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


804/804 [==============================] - 1s 766us/step - loss: 43794.3906 - accuracy: 0.4804
Epoch 2/100
804/804 [==============================] - 1s 793us/step - loss: 15321.9180 - accuracy: 0.5010
Epoch 3/100
804/804 [==============================] - 1s 757us/step - loss: 27200.7012 - accuracy: 0.4993
Epoch 4/100
804/804 [==============================] - 1s 776us/step - loss: 27404.9395 - accuracy: 0.5005
Epoch 5/100
793/804 [============================>.] - ETA: 0s - loss: 5456.0605 - accuracy: 0.5037
Epoch 5: saving model to checkpoints_attempt2_layer2/weights.005.hdf5
804/804 [==============================] - 1s 775us/step - loss: 5848.2241 - accuracy: 0.5036
Epoch 6/100
804/804 [==============================] - 1s 767us/step - loss: 21733.7754 - accuracy: 0.5224
Epoch 7/100
804/804 [==============================] - 1s 758us/step - loss: 16557.0469 - accuracy: 0.5390
Epoch 8/100
804/804 [==============================] - 1s 751us/step - loss: 15627.1650 - accuracy: 0.4976

Epoch 61/100
804/804 [==============================] - 1s 743us/step - loss: 0.6912 - accuracy: 0.5321
Epoch 62/100
804/804 [==============================] - 1s 747us/step - loss: 0.6912 - accuracy: 0.5321
Epoch 63/100
804/804 [==============================] - 1s 745us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 64/100
804/804 [==============================] - 1s 745us/step - loss: 0.6912 - accuracy: 0.5321
Epoch 65/100
739/804 [==========================>...] - ETA: 0s - loss: 0.6911 - accuracy: 0.5329
Epoch 65: saving model to checkpoints_attempt2_layer2/weights.065.hdf5
804/804 [==============================] - 1s 760us/step - loss: 0.6912 - accuracy: 0.5321
Epoch 66/100
804/804 [==============================] - 1s 746us/step - loss: 0.6912 - accuracy: 0.5321
Epoch 67/100
804/804 [==============================] - 1s 755us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 68/100
804/804 [==============================] - 1s 749us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 69/100

In [50]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 1.1510 - accuracy: 0.6482 - 215ms/epoch - 802us/step
Loss: 1.1510021686553955, Accuracy: 0.6481632590293884


### Deliverable 3: Attempt 3
#### Automated Neural Net Optimizer

#### Creating more bins for rare occurrences in columns : Expand CLASSIFICATION feature number from 6 to 13.
There is a reasonable break between CLASSIFICATION values C2800 (count 95) and C7100 (count 75)

Initially, choose not to use 13 CLASSIFICATION features, as there was little return on accuracy.
Use the binning in Attempt 1, i.e. 7 CLASSIFICATION features:
```
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64
```

The last two optimizations used 13 CLASSIFICATION features:
```
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      574
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
Name: CLASSIFICATION, dtype: int64
```


In [51]:
# read original dataframe with identifier fields already dropped
application_df = application0_df.copy()

# Determine which values to replace if counts are less than ..?
replace_class = list(classification_val_counts[classification_val_counts < 95].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      574
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
Name: CLASSIFICATION, dtype: int64

In [52]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
print(f"encode_df.shape: {encode_df.shape}")

# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
print(f"application_df.shape: {application_df.shape}")

# Split our preprocessed data into our features and target arrays
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Deliverable 2: Compile, Train and Evaluate the Model

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# # Define the checkpoint path and filenames
# os.makedirs("checkpoints_attempt2/",exist_ok=True)
# checkpoint_path = "checkpoints_opt2/weights.{epoch:04d}.hdf5"

# Define the checkpoint path and filenames
checkpoints_dir = f"checkpoints_attempt3_layer2"
print(checkpoints_dir)
os.makedirs(checkpoints_dir, exist_ok=True)
# checkpoint_path = "{checkpoints_dir}/weights.{epoch:03d}.hdf5"
checkpoint_path = os.path.join(checkpoints_dir, "weights.{epoch:03d}.hdf5")

# need numpy ceil function
import numpy as np

# calculate number of batches in 5 epochs
# default batch_size = 32 (https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit)
batch_size = 32
# round up number of batches
# (https://stackoverflow.com/questions/54159034/what-if-the-sample-size-is-not-divisible-by-batch-size-in-keras-model)
# Note: n_batches here is 804, which is the number displayed in the left column
# under each epoch in nn.fit() results display.
n_batches = int(np.ceil(len(X_train) / batch_size)) # np.ceil() return dtype float64
# n_batches = np.empty_like(n_batches, dtype=np.int32) # cast to int64
n_batches_per_5epochs = 5 * n_batches

# Create a callback that saves the model's weights every 5 epochs
# For non-deprecated way, pass an integer number of batches to save_freq
# (see https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint)
# For deprecated way, see https://stackoverflow.com/questions/59069058/save-model-every-10-epochs-tensorflow-keras-v2
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    # non-deprecated way:
    save_freq=n_batches_per_5epochs
    # deprecated way: (easier)
    ## save_freq='epoch',
    # period=5
)
# Note: Both ways create a checkpoint folder containing identical files:
# weights.0005.hdf5 weights.0010.hdf5 ... weights.0095.hdf5 weights.0100.hdf5

# Train the model
fit_model = nn.fit(X_train, y_train, epochs=100, callbacks=[cp_callback])

encode_df.shape: (34299, 56)
application_df.shape: (34299, 59)
Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_264 (Dense)           (None, 80)                4720      
                                                                 
 dense_265 (Dense)           (None, 30)                2430      
                                                                 
 dense_266 (Dense)           (None, 1)                 31        
                                                                 
Total params: 7,181
Trainable params: 7,181
Non-trainable params: 0
_________________________________________________________________
checkpoints_attempt3_layer2
Epoch 1/100


/Users/graemet/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/graemet/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  app.launch_new_instance()
/Users/graemet/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


804/804 [==============================] - 1s 883us/step - loss: 35691.2461 - accuracy: 0.4881
Epoch 2/100
804/804 [==============================] - 1s 835us/step - loss: 10505.5713 - accuracy: 0.5011
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 26424.8164 - accuracy: 0.5107
Epoch 4/100
804/804 [==============================] - 1s 892us/step - loss: 31299.1074 - accuracy: 0.4928
Epoch 5/100
790/804 [============================>.] - ETA: 0s - loss: 13312.5312 - accuracy: 0.4996
Epoch 5: saving model to checkpoints_attempt3_layer2/weights.005.hdf5
804/804 [==============================] - 1s 846us/step - loss: 13085.2529 - accuracy: 0.5001
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 28608.1445 - accuracy: 0.5117
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 9200.3311 - accuracy: 0.4923
Epoch 8/100
804/804 [==============================] - 1s 827us/step - loss: 11900.6074 - accuracy: 0.5241
Epoc

804/804 [==============================] - 1s 805us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 62/100
804/804 [==============================] - 1s 760us/step - loss: 0.6912 - accuracy: 0.5321
Epoch 63/100
804/804 [==============================] - 1s 751us/step - loss: 0.6913 - accuracy: 0.5321
Epoch 64/100
804/804 [==============================] - 1s 748us/step - loss: 0.6913 - accuracy: 0.5321
Epoch 65/100
738/804 [==========================>...] - ETA: 0s - loss: 0.6911 - accuracy: 0.5330
Epoch 65: saving model to checkpoints_attempt3_layer2/weights.065.hdf5
804/804 [==============================] - 1s 759us/step - loss: 0.6912 - accuracy: 0.5321
Epoch 66/100
804/804 [==============================] - 1s 758us/step - loss: 0.6912 - accuracy: 0.5321
Epoch 67/100
804/804 [==============================] - 1s 760us/step - loss: 0.6912 - accuracy: 0.5321
Epoch 68/100
804/804 [==============================] - 1s 753us/step - loss: 0.6912 - accuracy: 0.5321
Epoch 69/100
804/804 [===

In [53]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.8807 - accuracy: 0.6959 - 223ms/epoch - 831us/step
Loss: 0.880744218826294, Accuracy: 0.6958600878715515


### Deliverable 3: Attempt 4
#### Automated Neural Net Optimizer

There is negligible difference between 7 and 13 `CLASSIFICATION` categories, so choose 7.
```
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64
```




In [55]:
# read original dataframe with identifier fields already dropped
application_df = application0_df.copy()

# Determine which values to replace if counts are less than ..?
replace_class = list(classification_val_counts[classification_val_counts < 528].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [56]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
print(f'One-Hot encoded DataFrame shape: {encode_df.shape}')
# encode_df.head()

# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
print(f'Merged/dropped application_df shape: {application_df.shape}')
# application_df.head()

# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

One-Hot encoded DataFrame shape: (34299, 50)
Merged/dropped application_df shape: (34299, 53)


/Users/graemet/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/graemet/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/Users/graemet/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


In [57]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])

# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu', 'tanh', 'sigmoid']) # 'relu','tanh','sigmoid'
    
    # Allow kerastuner to decide number of neurons in first layer ## TRY up to 100 ##
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=20,
        max_value=100,
        step=5), activation=activation, input_dim=number_input_features))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=4,
            max_value=30, ## TRY up to 30 ##
            step=2),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [58]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    seed=78,
    overwrite=True,
    max_epochs=20,
    hyperband_iterations=8,
    directory='anno030')

# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 240 Complete [00h 00m 17s]
val_accuracy: 0.726064145565033

Best val_accuracy So Far: 0.7295626997947693
Total elapsed time: 00h 30m 52s
INFO:tensorflow:Oracle triggered exit


In [59]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
print(best_hyper.values)

# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

{'activation': 'relu', 'first_units': 65, 'num_layers': 5, 'units_0': 8, 'units_1': 4, 'units_2': 10, 'units_3': 6, 'units_4': 8, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0042'}
268/268 - 0s - loss: 0.5537 - accuracy: 0.7296 - 279ms/epoch - 1ms/step
Loss: 0.553713858127594, Accuracy: 0.7295626997947693


In [60]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# need numpy ceil function to calculate number of batches
import numpy as np

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])

# {'activation': 'relu', 'first_units': 65, 
# 'num_layers': 5, 'units_0': 8, 'units_1': 4, 'units_2': 10, 'units_3': 6, 'units_4': 8, 
# 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0042'}
# 268/268 - 0s - loss: 0.5537 - accuracy: 0.7296 - 279ms/epoch - 1ms/step
# Loss: 0.553713858127594, Accuracy: 0.7295626997947693

# taken from automated neural network optimizer above
hidden_nodes_layer1 = 65
hidden_nodes_layer2 = 8
hidden_nodes_layer3 = 4
hidden_nodes_layer4 = 10
hidden_nodes_layer5 = 6
hidden_nodes_layer6 = 8

nn = tf.keras.models.Sequential()
activation = "relu"

# First hidden layer (first_units)
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation=activation)
)

# Second hidden layer (units_0)
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation=activation))

# Third hidden layer (units_1)
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation=activation))

# Fourth hidden layer (units_2)
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation=activation))

# Fifth hidden layer (units_3)
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation=activation))

# Sixth hidden layer (units_4)
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer6, activation=activation))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
print(nn.summary())

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

#### Use checkpoints

# Define the checkpoint path and filenames
checkpoints_dir = f"checkpoints_attempt3_layer6"
print(checkpoints_dir)
os.makedirs(checkpoints_dir, exist_ok=True)
# checkpoint_path = "{checkpoints_dir}/weights.{epoch:03d}.hdf5"
checkpoint_path = os.path.join(checkpoints_dir, "weights.{epoch:03d}.hdf5")

# calculate number of batches in 5 epochs
# default batch_size = 32 (https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit)
batch_size = 32
# round up number of batches
# (https://stackoverflow.com/questions/54159034/what-if-the-sample-size-is-not-divisible-by-batch-size-in-keras-model)
# Note: n_batches here is 804, which is the number displayed in the left column
# under each epoch in nn.fit() results display.
n_batches = int(np.ceil(len(X_train) / batch_size)) # np.ceil() return dtype float64
# n_batches = np.empty_like(n_batches, dtype=np.int32) # cast to int64
n_batches_per_5epochs = 5 * n_batches

# Create a callback that saves the model's weights every 5 epochs
# For non-deprecated way, pass an integer number of batches to save_freq
# (see https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint)
# For deprecated way, see https://stackoverflow.com/questions/59069058/save-model-every-10-epochs-tensorflow-keras-v2
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    # non-deprecated way:
    save_freq=n_batches_per_5epochs
    # deprecated way: (easier)
    ## save_freq='epoch',
    # period=5
)
# Note: Both ways create a checkpoint folder containing identical files:
# weights.0005.hdf5 weights.0010.hdf5 ... weights.0095.hdf5 weights.0100.hdf5


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 65)                3445      
                                                                 
 dense_8 (Dense)             (None, 8)                 528       
                                                                 
 dense_9 (Dense)             (None, 4)                 36        
                                                                 
 dense_10 (Dense)            (None, 10)                50        
                                                                 
 dense_11 (Dense)            (None, 6)                 66        
                                                                 
 dense_12 (Dense)            (None, 8)                 56        
                                                                 
 dense_13 (Dense)            (None, 1)                

In [61]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=100, callbacks=[cp_callback])


Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 90.4282 - accuracy: 0.4971
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6914 - accuracy: 0.5321
Epoch 3/100
804/804 [==============================] - 1s 920us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 5/100
801/804 [============================>.] - ETA: 0s - loss: 0.6911 - accuracy: 0.5321
Epoch 5: saving model to checkpoints_attempt3_layer6/weights.005.hdf5
804/804 [==============================] - 1s 907us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 6/100
804/804 [==============================] - 1s 912us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 7/100
804/804 [==============================] - 1s 844us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 8/100
804/804 [==============================] - 1s 844us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 9/100
804/804 [=====

804/804 [==============================] - 1s 858us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 62/100
804/804 [==============================] - 1s 854us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 63/100
804/804 [==============================] - 1s 851us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 64/100
804/804 [==============================] - 1s 836us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 65/100
802/804 [============================>.] - ETA: 0s - loss: 0.6911 - accuracy: 0.5321
Epoch 65: saving model to checkpoints_attempt3_layer6/weights.065.hdf5
804/804 [==============================] - 1s 834us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 66/100
804/804 [==============================] - 1s 878us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 67/100
804/804 [==============================] - 1s 822us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 68/100
804/804 [==============================] - 1s 809us/step - loss: 0.6911 - accuracy: 0.5321
Epoch 69/100
804/804 [===

In [62]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6918 - accuracy: 0.5334 - 240ms/epoch - 896us/step
Loss: 0.6917797327041626, Accuracy: 0.5334110856056213


## Addendum

#### Results
A variety of automated neural net optimizer parameters were chosen, as tuner.search was capped at 60 trials
before NFO:tensorflow:Oracle triggered exit.

| directory | num features| activation | loss | accuracy | input hidden 1 | hidden 2 | hidden 3 | hidden 4 | hidden 5 | hidden 6 |
| --- | --- | --- | :--- | :--- | --- | --- | --- | --- | --- | --- |
| anno001 | 43 | relu | 0.5513805150985718 | 0.7295626997947693 | 110 | 60 | | | |
| anno002 | 43 | relu | 0.5496115684509277 | 0.7292128205299377 | 100 | 60 | 60 | | |
| anno003 | 43 | relu | 0.5498868823051453 | 0.7289795875549316 | 130 | 80 | 60 | | |
| anno004 | 43 | relu | 0.5506145358085632 | 0.7292128205299377 | 110 | 80 | 80 | | | 
| anno005 | 43 | relu | 0.5495125651359558 | 0.7297959327697754 | 130 | 80 | 60 | 60 | |
| anno006 | 43 | relu | 0.5538275837898254 | 0.7289795875549316 | 130 | 80 | 80 | 80 | |
| anno007 | 43 | relu | 0.5492367148399353 | 0.7294460535049438 | 110 | 60 | 80 | 80 | 80 |
| anno008 | 43 | tanh | 0.5576893091201782 | 0.728396475315094  | 130 | 60 | 80 | 60 |  |
| anno009 | 43 | sigm | 0.5577203631401062 | 0.7257142663002014 | 110 | 60 | 60 | 80 |  |
| anno010 | 43 | relu | 0.5542266964912415 | 0.728863000869751  | 120 | 70 | 70 | 90 |  |
| anno011 | 49 | relu | 0.5495337843894958 | 0.7302623987197876 | 140 | 80 | 80 | 60 |  |
| anno012 | 49 | relu | 0.5478928089141846 | 0.729912519454956  | 150 | 70 | 70 | 90 |  |
| anno013 | 49 | relu | 0.5482209324836731 | 0.7306122183799744 | 150 | 90 | 110 |  |  |
| anno014 | 49 | relu | 0.5516106486320496 | 0.7303789854049683 | 150 | 100 | 120 |  |  |
| anno015 | 49 | tanh | 0.5476542115211487 | 0.7304956316947937 | 150 | 80 | 100 |  |  |
| anno016 | 49 | sigm | 0.5594959855079651 | 0.7287463545799255 | 150 | 120 | 120 |  |  |
| anno017 | 49 | relu | 0.550683319568634  | 0.72967928647995   |  28 |  16 |  14 |  |  |
| anno018 | 49 | tanh | 0.5494146347045898 | 0.731195330619812  |  80 |  16 |  12 | 16 |  |
| anno019 | 49 | tanh | 0.5491611361503601 | 0.7309620976448059 |  85 |  24 |  10 | 28 | 12 | 20 |
| anno020 | 49 | tanh | 0.5524029731750488 | 0.72967928647995   |   6 |  26 |  28 | | | |
| anno021 | 49 | tanh | 0.5521022081375122 | 0.7313119769096375 |  75 |  16 |  20 | 14 | 30 | 14 |
| anno022 | 49 | relu | 0.5473343729972839 | 0.7302623987197876 |  55 |   8 |  16 | 12 | 26 | 24 |

Note: The best input layers have 2-3 times the number of neurons as features.